In [ ]:
%%javascript

require(['widgets/js/widget', 'widgets/js/manager', 'jquery', 'underscore'], function(widget, manager, $, _) {
    var GoogleMapsView = widget.DOMWidgetView.extend({
        initialize: function() {
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                }
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'], 
                    function() {}, 
                    function() {});
            }
        },
        
        render: function() {
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $(
                        '<div style="height: ' + this.model.get('height') + '; width: ' + this.model.get('width') + ';"></div>');
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {lat: this.model.get('lat'), lng: this.model.get('lng')},
                        zoom: this.model.get('zoom')
                    });
                    
                    this.listenTo(this.model, 'change:height', _.bind(function() {
                        this.$map.height(this.model.get('height'));
                        google.maps.event.trigger(this.map, 'resize');
                    }, this));
                    this.listenTo(this.model, 'change:width', _.bind(function() {
                        this.$map.width(this.model.get('width'));
                        google.maps.event.trigger(this.map, 'resize');
                    }, this));
                    
                    google.maps.event.addListener(this.map, 'bounds_changed', _.bind(this.syncFromMap, this));
                    
                    this.listenTo(this.model, 'change:lat', _.bind(this.syncFromModel, this));
                    this.listenTo(this.model, 'change:lng', _.bind(this.syncFromModel, this));
                    this.listenTo(this.model, 'change:zoom', _.bind(this.syncFromModel, this));
                    
                    this.listenTo(this.model, 'msg:custom', _.bind(this.handleMessage, this));
                }, this));
            }, this));
        },
        
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            this.model.save_changes();
        },
        
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },
        
        handleMessage: function(payload) {
            var params = payload.params || [];
            this[payload.method].apply(this, params);
        },
        
        addLayer: function() {
            console.log(arguments);
            //console.log(arguments[0].mapid);
            
            var mapid = arguments[0].mapid
            var token = arguments[0].token
            var name = arguments[0].name
            var visible = arguments[0].visible
            
            var eeMapOptions = {
                getTileUrl: function(tile, zoom) {
                    var url = ['https://earthengine.googleapis.com/map',
                               mapid, zoom, tile.x, tile.y].join("/");
                    url += '?token=' + token
                    return url;
                },
                tileSize: new window.google.maps.Size(256, 256),
                opacity: visible ? 1.0 : 0.0,
            };
            
            // Create the overlay map type
            var mapType = new window.google.maps.ImageMapType(eeMapOptions);
                
            // Overlay the Earth Engine generated layer
            this.map.overlayMapTypes.push(mapType);
        }
    });
    
    manager.WidgetManager.register_widget_view('GoogleMapsView', GoogleMapsView);
});

In [ ]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    _view_name = traitlets.Unicode('GoogleMapsView', sync=True)
    lat = traitlets.CFloat(0, sync=True)
    lng = traitlets.CFloat(0, sync=True)
    zoom = traitlets.CInt(2, sync=True)
    
    height = traitlets.CUnicode('300px', sync=True)
    width = traitlets.CUnicode('400px', sync=True)
        
    def addLayer(self, image, vis_params):
        mapid = image.getMapId(vis_params)
        self.send({'method': 'addLayer',
                   'params': [{
                        'mapid':mapid['mapid'], 
                        'token':mapid['token'],
                        'name':"dummy",
                        'visible':True
                    }]
                  })

In [ ]:
from IPython.display import display
import ee

l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2013-05-01', '2013-05-15').mosaic()
dem = ee.Image('USGS/SRTMGL1_003')

vis_params_dem = {'min':0, 'max':3000}
vis_params_true = {'min':0, 'max':0.3, 'bands':'B4,B3,B2'}
vis_params_false = {'min':0, 'max':0.3, 'bands':'B5,B4,B3'}

# Overview map.
map1 = GoogleMapsView(lng=-119.2, lat=36.3, zoom=4)
map1.height = '200px'
map1.width = '600px'

# SRTM
map2 = GoogleMapsView(zoom=6)
map2.height = map1.height
map2.width = map1.width

# Landsat 8 True Color
map3 = GoogleMapsView()
map3.height = map1.height
map3.width = map1.width

# Landsat 8 False Color
map4 = GoogleMapsView()
map4.height = map1.height
map4.width = map1.width

In [ ]:
# Link up the interactive content
widgets.jslink((map1, 'lat'), (map2, 'lat'))
widgets.jslink((map1, 'lat'), (map3, 'lat'))
widgets.jslink((map1, 'lat'), (map4, 'lat'))
widgets.jslink((map1, 'lng'), (map2, 'lng'))
widgets.jslink((map1, 'lng'), (map3, 'lng'))
widgets.jslink((map1, 'lng'), (map4, 'lng'))
widgets.jslink((map2, 'zoom'), (map3, 'zoom'))
widgets.jslink((map2, 'zoom'), (map4, 'zoom'))

In [ ]:
display(map1, map2, map3, map4)
# Overlay some Earth Engine data.
map2.addLayer(image=dem, vis_params=vis_params_dem)
map3.addLayer(image=l8, vis_params=vis_params_true)
map4.addLayer(image=l8, vis_params=vis_params_false)

In [ ]:
(map4.lng, map4.lat)